인터 폴레이션을 검증하여 $R^2$ 와 $MSE$ 를 검증하는 파일입니다.

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
#BASE_WORKING_DIR = "/content/gdrive/My Drive/Colab Notebooks/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.csv
result.gsheet
test_cond.csv
train_ta_hm.csv

===== Convert

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.externals import joblib

# interpolation 함수
import scipy.interpolate as spi

# pickle
import pickle

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

In [ ]:
def stringToDatetime(df, colName='mea_ddhr'):
  """
  describe : string 을 datetime으로 변경한다.
  input = pd.Dataframe
  output = pd.Dataframe
  """
  if type(df[colName][0]) != str: # 이미 자료형이 datetime인 경우 return 한다.
    return df
  if len(df[colName][0].split(' ')[1].split(':')) == 2: # 날짜 정보에 초에 대한 정보가 없을 경우 초를 붙여주고 datetime으로 변경한다.
    df[colName] = df[colName].apply(lambda x : x + ':00' if x != 'nan' else np.nan)
  df[colName] = df[colName].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if x != 'nan' else np.nan)
  return df

def createDateTime(df, colName='mea_ddhr'):
  """
  describe : datetime의 정보에서 date 정보와 0시 기준 몇 분인지를 나타내는 time을 분리하여 추가한다.
  input = pd.Dataframe
  output = pd.Dataframe
  """
  if type(df[colName][0]) == "str":
    df[colName] = df[colName].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"))
  df = stringToDatetime(df, colName)
  print(df)
  df['date'] = df[colName].apply(lambda x : x.date())
  df['time'] = df[colName].apply(lambda x : np.float64(x.hour * 60 + x.minute))
  return df

In [ ]:
path_dir = os.path.join(DATA_EXPORT_PATH, "dangjin_aws")
file_list = os.listdir(path_dir)
AWS_PATH = []
for file in file_list:
  if 'MI' in file:
    AWS_PATH.append(os.path.join(path_dir, file))
AWS_PATH

['/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_12.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_10.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_08.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_07.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_05.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_03.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_01.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2016_12.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2016_10.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_6

In [ ]:
AWS_DATA = pd.DataFrame(columns=["stn_id", "mea_ddhr", "ta", "rn", "rn_onehot", "wd", "ws", "lp", "ap", "hm", "sr", "ss"])

for path in AWS_PATH:
  data = pd.read_csv(path, encoding = 'cp949')
  data.columns = ["stn_id", "mea_ddhr", "ta", "rn", "rn_onehot", "wd", "ws", "lp", "ap", "hm", "sr", "ss"]
  AWS_DATA = AWS_DATA.append(data)
  print("{} 파일 작업중... ".format(path))

AWS_DATA.reset_index(inplace=True)
AWS_DATA.drop(columns=['index'], inplace=True)
AWS_DATA

/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_12.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_10.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_08.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_07.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_05.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_03.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2017_01.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2016_12.csv 파일 작업중... 
/content/gdrive/My Drive/weather_bigcontest/4_code/data/export/dangjin_aws/AWS_616_MI_2016_10.csv 파일 작업중... 
/content/gdrive/My 

,stn_id,mea_ddhr,ta,rn,rn_onehot,wd,ws,lp,ap,hm,sr,ss
0,616,2017-12-01 00:01,-1.3,0.0,0,343.9,2.5,NaN,NaN,62.1,NaN,NaN
1,616,2017-12-01 00:02,-1.3,0.0,0,337.9,1.4,NaN,NaN,62.1,NaN,NaN
2,616,2017-12-01 00:03,-1.3,0.0,0,349.9,2.7,NaN,NaN,62.2,NaN,NaN
3,616,2017-12-01 00:04,-1.3,0.0,0,338.1,3.0,NaN,NaN,62.3,NaN,NaN
4,616,2017-12-01 00:05,-1.3,0.0,0,339.0,2.2,NaN,NaN,62.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2135195,616,2016-03-31 23:56,7.0,0.0,0,0.0,0.1,NaN,NaN,40.9,NaN,NaN
2135196,616,2016-03-31 23:57,7.2,0.0,0,155.2,0.3,NaN,NaN,39.2,NaN,NaN
2135197,616,2016-03-31 23:58,7.3,0.0,0,175.1,0.2,NaN,NaN,38.1,NaN,NaN
2135198,616,2016-03-31 23:59,7.1,0.0,0,196.9,0.3,NaN,NaN,39.2,NaN,NaN


In [ ]:
AWS_DATA['mea_ddhr'] = AWS_DATA['mea_ddhr'].apply(lambda x : x + ':00')
AWS_DATA['mea_ddhr'] = AWS_DATA['mea_ddhr'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
AWS_DATA = createDateTime(AWS_DATA)
AWS_DATA

        stn_id            mea_ddhr   ta   rn rn_onehot  ...  lp  ap    hm  sr  ss
0          616 2017-12-01 00:01:00 -1.3  0.0         0  ... NaN NaN  62.1 NaN NaN
1          616 2017-12-01 00:02:00 -1.3  0.0         0  ... NaN NaN  62.1 NaN NaN
2          616 2017-12-01 00:03:00 -1.3  0.0         0  ... NaN NaN  62.2 NaN NaN
3          616 2017-12-01 00:04:00 -1.3  0.0         0  ... NaN NaN  62.3 NaN NaN
4          616 2017-12-01 00:05:00 -1.3  0.0         0  ... NaN NaN  62.4 NaN NaN
...        ...                 ...  ...  ...       ...  ...  ..  ..   ...  ..  ..
2135195    616 2016-03-31 23:56:00  7.0  0.0         0  ... NaN NaN  40.9 NaN NaN
2135196    616 2016-03-31 23:57:00  7.2  0.0         0  ... NaN NaN  39.2 NaN NaN
2135197    616 2016-03-31 23:58:00  7.3  0.0         0  ... NaN NaN  38.1 NaN NaN
2135198    616 2016-03-31 23:59:00  7.1  0.0         0  ... NaN NaN  39.2 NaN NaN
2135199    616 2016-04-01 00:00:00  6.8  0.0         0  ... NaN NaN  41.2 NaN NaN

[2135200 rows x

,stn_id,mea_ddhr,ta,rn,rn_onehot,wd,ws,lp,ap,hm,sr,ss,date,time
0,616,2017-12-01 00:01:00,-1.3,0.0,0,343.9,2.5,NaN,NaN,62.1,NaN,NaN,2017-12-01,1.0
1,616,2017-12-01 00:02:00,-1.3,0.0,0,337.9,1.4,NaN,NaN,62.1,NaN,NaN,2017-12-01,2.0
2,616,2017-12-01 00:03:00,-1.3,0.0,0,349.9,2.7,NaN,NaN,62.2,NaN,NaN,2017-12-01,3.0
3,616,2017-12-01 00:04:00,-1.3,0.0,0,338.1,3.0,NaN,NaN,62.3,NaN,NaN,2017-12-01,4.0
4,616,2017-12-01 00:05:00,-1.3,0.0,0,339.0,2.2,NaN,NaN,62.4,NaN,NaN,2017-12-01,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135195,616,2016-03-31 23:56:00,7.0,0.0,0,0.0,0.1,NaN,NaN,40.9,NaN,NaN,2016-03-31,1436.0
2135196,616,2016-03-31 23:57:00,7.2,0.0,0,155.2,0.3,NaN,NaN,39.2,NaN,NaN,2016-03-31,1437.0
2135197,616,2016-03-31 23:58:00,7.3,0.0,0,175.1,0.2,NaN,NaN,38.1,NaN,NaN,2016-03-31,1438.0
2135198,616,2016-03-31 23:59:00,7.1,0.0,0,196.9,0.3,NaN,NaN,39.2,NaN,NaN,2016-03-31,1439.0


In [ ]:
AWS_DATA = AWS_DATA[['mea_ddhr', 'date', 'time', 'ta', 'hm', 'ws', 'rn']]
AWS_DATA

,mea_ddhr,date,time,ta,hm,ws,rn
0,2017-12-01 00:01:00,2017-12-01,1.0,-1.3,62.1,2.5,0.0
1,2017-12-01 00:02:00,2017-12-01,2.0,-1.3,62.1,1.4,0.0
2,2017-12-01 00:03:00,2017-12-01,3.0,-1.3,62.2,2.7,0.0
3,2017-12-01 00:04:00,2017-12-01,4.0,-1.3,62.3,3.0,0.0
4,2017-12-01 00:05:00,2017-12-01,5.0,-1.3,62.4,2.2,0.0
...,...,...,...,...,...,...,...
2135195,2016-03-31 23:56:00,2016-03-31,1436.0,7.0,40.9,0.1,0.0
2135196,2016-03-31 23:57:00,2016-03-31,1437.0,7.2,39.2,0.3,0.0
2135197,2016-03-31 23:58:00,2016-03-31,1438.0,7.3,38.1,0.2,0.0
2135198,2016-03-31 23:59:00,2016-03-31,1439.0,7.1,39.2,0.3,0.0


In [ ]:
date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()

AWS_DATA = AWS_DATA[(date_start <= AWS_DATA['date']) & (AWS_DATA['date'] <= date_end)]
AWS_DATA

,mea_ddhr,date,time,ta,hm,ws,rn
0,2017-12-01 00:01:00,2017-12-01,1.0,-1.3,62.1,2.5,0.0
1,2017-12-01 00:02:00,2017-12-01,2.0,-1.3,62.1,1.4,0.0
2,2017-12-01 00:03:00,2017-12-01,3.0,-1.3,62.2,2.7,0.0
3,2017-12-01 00:04:00,2017-12-01,4.0,-1.3,62.3,3.0,0.0
4,2017-12-01 00:05:00,2017-12-01,5.0,-1.3,62.4,2.2,0.0
...,...,...,...,...,...,...,...
2090559,2020-03-31 23:56:00,2020-03-31,1436.0,7.3,67.4,0.8,0.0
2090560,2020-03-31 23:57:00,2020-03-31,1437.0,7.3,67.2,0.7,0.0
2090561,2020-03-31 23:58:00,2020-03-31,1438.0,7.4,67.4,0.9,0.0
2090562,2020-03-31 23:59:00,2020-03-31,1439.0,7.4,67.2,0.8,0.0


In [ ]:
AWS_DATA = AWS_DATA[AWS_DATA['time']%10 == 0]
AWS_DATA

,mea_ddhr,date,time,ta,hm,ws,rn
9,2017-12-01 00:10:00,2017-12-01,10.0,-1.3,62.8,2.3,0.0
19,2017-12-01 00:20:00,2017-12-01,20.0,-1.3,62.3,2.9,0.0
29,2017-12-01 00:30:00,2017-12-01,30.0,-1.3,62.3,2.0,0.0
39,2017-12-01 00:40:00,2017-12-01,40.0,-1.6,62.7,2.6,0.0
49,2017-12-01 00:50:00,2017-12-01,50.0,-1.9,63.4,2.7,0.0
...,...,...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,7.5,67.1,0.3,0.0
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,7.4,67.3,1.0,0.0
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,7.3,67.8,0.6,0.0
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,7.1,68.9,0.9,0.0


In [ ]:
# interpolation dictionary 설명서를 load 한다.
IPO_DICT_READ_ME = pd.read_csv(os.path.join(MODEL_PATH, "IPO_DICT_READ_ME.csv"))
IPO_DICT_READ_ME = IPO_DICT_READ_ME.fillna(" ")
IPO_DICT_READ_ME

,WHERE :,dangjin,sinpyeong,seosan,songsan,songak
0,TARGET :,ta,ta,ta,ta,ta
1,,hm,ws,hm,hm,hm
2,,ws,rn,ws,ws,ws
3,,rn,lp,rn,rn,rn
4,,,ap,dew,,
5,,,,sp,,
6,,,,lp,,
7,,,,ap,,
8,,,,sr,,
9,,,,ss,,


In [ ]:
# interpolation dictionary 를 load 한다.
import pickle
ipo_dict = joblib.load(os.path.join(MODEL_PATH, "ipo_dict.pkl"))
ipo_dict.keys()

dict_keys([('dangjin', 'ta', datetime.date(2016, 4, 1)), ('dangjin', 'ta', datetime.date(2016, 4, 2)), ('dangjin', 'ta', datetime.date(2016, 4, 3)), ('dangjin', 'ta', datetime.date(2016, 4, 4)), ('dangjin', 'ta', datetime.date(2016, 4, 5)), ('dangjin', 'ta', datetime.date(2016, 4, 6)), ('dangjin', 'ta', datetime.date(2016, 4, 7)), ('dangjin', 'ta', datetime.date(2016, 4, 8)), ('dangjin', 'ta', datetime.date(2016, 4, 9)), ('dangjin', 'ta', datetime.date(2016, 4, 10)), ('dangjin', 'ta', datetime.date(2016, 4, 11)), ('dangjin', 'ta', datetime.date(2016, 4, 12)), ('dangjin', 'ta', datetime.date(2016, 4, 13)), ('dangjin', 'ta', datetime.date(2016, 4, 14)), ('dangjin', 'ta', datetime.date(2016, 4, 15)), ('dangjin', 'ta', datetime.date(2016, 4, 16)), ('dangjin', 'ta', datetime.date(2016, 4, 17)), ('dangjin', 'ta', datetime.date(2016, 4, 18)), ('dangjin', 'ta', datetime.date(2016, 4, 19)), ('dangjin', 'ta', datetime.date(2016, 4, 20)), ('dangjin', 'ta', datetime.date(2016, 4, 21)), ('dangjin',

In [ ]:
def pred_weather(df, target):
  df[target + '-pred'] = 0

  date_start = datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date_end = datetime.strptime("2020-03-31 00:00:00", '%Y-%m-%d %H:%M:%S').date()
  date = date_start
  
  while date <= date_end:
    if date.month == 1 and date.day == 1:
      print('{} 작업중...'.format(date))
    df_date = df[df['date'] == date]
    key = ('dangjin', target, date)
    ipo = ipo_dict[key]
    df_date[target + '-pred'] = df_date['time'].apply(lambda x : spi.splev(x, ipo))
    df.loc[df_date.index] = df_date
    date += timedelta(days=1)
  return df

In [ ]:
def calc_R(df, target):
  mean = np.mean(df[target])
  diff_mean_squre = np.dot((df[target] - mean), (df[target] - mean))
  diff_pred_squre = np.dot((df[target] - df[target+'-pred']), (df[target] - df[target+'-pred']))
  print(np.sqrt(diff_pred_squre/len(df)))
  return 1 - diff_pred_squre/diff_mean_squre

In [ ]:
AWS_DATA_TA = AWS_DATA[['mea_ddhr', 'date', 'time', 'ta']]
AWS_DATA_TA = AWS_DATA_TA.dropna()
AWS_DATA_TA

,mea_ddhr,date,time,ta
9,2017-12-01 00:10:00,2017-12-01,10.0,-1.3
19,2017-12-01 00:20:00,2017-12-01,20.0,-1.3
29,2017-12-01 00:30:00,2017-12-01,30.0,-1.3
39,2017-12-01 00:40:00,2017-12-01,40.0,-1.6
49,2017-12-01 00:50:00,2017-12-01,50.0,-1.9
...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,7.5
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,7.4
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,7.3
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,7.1


In [ ]:
AWS_DATA_TA = pred_weather(AWS_DATA_TA, 'ta')
AWS_DATA_TA

2017-01-01 작업중...
2018-01-01 작업중...
2019-01-01 작업중...
2020-01-01 작업중...


,mea_ddhr,date,time,ta,ta-pred
9,2017-12-01 00:10:00,2017-12-01,10.0,-1.3,-1.433333
19,2017-12-01 00:20:00,2017-12-01,20.0,-1.3,-1.566667
29,2017-12-01 00:30:00,2017-12-01,30.0,-1.3,-1.700000
39,2017-12-01 00:40:00,2017-12-01,40.0,-1.6,-1.833333
49,2017-12-01 00:50:00,2017-12-01,50.0,-1.9,-1.966667
...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,7.5,7.700000
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,7.4,7.800000
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,7.3,7.900000
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,7.1,8.000000


In [ ]:
calc_R(AWS_DATA_TA, 'ta')

0.37260611951576506


0.9987073721182916

In [ ]:
AWS_DATA_TA = pred_weather(AWS_DATA_TA, 'ta')
AWS_DATA_TA

2017-01-01 작업중...
2018-01-01 작업중...
2019-01-01 작업중...
2020-01-01 작업중...


,mea_ddhr,date,time,ta,ta-pred
9,2017-12-01 00:10:00,2017-12-01,10.0,-1.3,-1.433333
19,2017-12-01 00:20:00,2017-12-01,20.0,-1.3,-1.566667
29,2017-12-01 00:30:00,2017-12-01,30.0,-1.3,-1.700000
39,2017-12-01 00:40:00,2017-12-01,40.0,-1.6,-1.833333
49,2017-12-01 00:50:00,2017-12-01,50.0,-1.9,-1.966667
...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,7.5,7.700000
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,7.4,7.800000
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,7.3,7.900000
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,7.1,8.000000


In [ ]:
AWS_DATA_HM = AWS_DATA[['mea_ddhr', 'date', 'time', 'hm']]
AWS_DATA_HM = AWS_DATA_HM.dropna()
AWS_DATA_HM

,mea_ddhr,date,time,hm
9,2017-12-01 00:10:00,2017-12-01,10.0,62.8
19,2017-12-01 00:20:00,2017-12-01,20.0,62.3
29,2017-12-01 00:30:00,2017-12-01,30.0,62.3
39,2017-12-01 00:40:00,2017-12-01,40.0,62.7
49,2017-12-01 00:50:00,2017-12-01,50.0,63.4
...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,67.1
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,67.3
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,67.8
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,68.9


In [ ]:
AWS_DATA_HM = pred_weather(AWS_DATA_HM, 'hm')
AWS_DATA_HM

2017-01-01 작업중...
2018-01-01 작업중...
2019-01-01 작업중...
2020-01-01 작업중...


,mea_ddhr,date,time,hm,hm-pred
9,2017-12-01 00:10:00,2017-12-01,10.0,62.8,62.333333
19,2017-12-01 00:20:00,2017-12-01,20.0,62.3,62.566667
29,2017-12-01 00:30:00,2017-12-01,30.0,62.3,62.800000
39,2017-12-01 00:40:00,2017-12-01,40.0,62.7,63.033333
49,2017-12-01 00:50:00,2017-12-01,50.0,63.4,63.266667
...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,67.1,64.700000
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,67.3,63.850000
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,67.8,63.000000
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,68.9,62.150000


In [ ]:
calc_R(AWS_DATA_HM, 'hm')

1.8439857025163537


0.9906366884772652

In [ ]:
AWS_DATA_WS = AWS_DATA[['mea_ddhr', 'date', 'time', 'ws']]
AWS_DATA_WS = AWS_DATA_WS.dropna()
AWS_DATA_WS

,mea_ddhr,date,time,ws
9,2017-12-01 00:10:00,2017-12-01,10.0,2.3
19,2017-12-01 00:20:00,2017-12-01,20.0,2.9
29,2017-12-01 00:30:00,2017-12-01,30.0,2.0
39,2017-12-01 00:40:00,2017-12-01,40.0,2.6
49,2017-12-01 00:50:00,2017-12-01,50.0,2.7
...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,0.3
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,1.0
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,0.6
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,0.9


In [ ]:
AWS_DATA_WS = pred_weather(AWS_DATA_WS, 'ws')
AWS_DATA_WS

2017-01-01 작업중...
2018-01-01 작업중...
2019-01-01 작업중...
2020-01-01 작업중...


,mea_ddhr,date,time,ws,ws-pred
9,2017-12-01 00:10:00,2017-12-01,10.0,2.3,2.833333
19,2017-12-01 00:20:00,2017-12-01,20.0,2.9,2.866667
29,2017-12-01 00:30:00,2017-12-01,30.0,2.0,2.900000
39,2017-12-01 00:40:00,2017-12-01,40.0,2.6,2.933333
49,2017-12-01 00:50:00,2017-12-01,50.0,2.7,2.966667
...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,0.3,0.766667
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,1.0,0.700000
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,0.6,0.633333
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,0.9,0.566667


In [ ]:
calc_R(AWS_DATA_WS, 'ws')

0.5937815710254837


0.7656917854058483

In [ ]:
AWS_DATA_RN = AWS_DATA[['mea_ddhr', 'date', 'time', 'rn']]
AWS_DATA_RN = AWS_DATA_RN.dropna()
AWS_DATA_RN

,mea_ddhr,date,time,rn
9,2017-12-01 00:10:00,2017-12-01,10.0,0.0
19,2017-12-01 00:20:00,2017-12-01,20.0,0.0
29,2017-12-01 00:30:00,2017-12-01,30.0,0.0
39,2017-12-01 00:40:00,2017-12-01,40.0,0.0
49,2017-12-01 00:50:00,2017-12-01,50.0,0.0
...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,0.0
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,0.0
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,0.0
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,0.0


In [ ]:
AWS_DATA_RN = pred_weather(AWS_DATA_RN, 'rn')
AWS_DATA_RN

2017-01-01 작업중...
2018-01-01 작업중...
2019-01-01 작업중...
2020-01-01 작업중...


,mea_ddhr,date,time,rn,rn-pred
9,2017-12-01 00:10:00,2017-12-01,10.0,0.0,0.0
19,2017-12-01 00:20:00,2017-12-01,20.0,0.0,0.0
29,2017-12-01 00:30:00,2017-12-01,30.0,0.0,0.0
39,2017-12-01 00:40:00,2017-12-01,40.0,0.0,0.0
49,2017-12-01 00:50:00,2017-12-01,50.0,0.0,0.0
...,...,...,...,...,...
2090523,2020-03-31 23:20:00,2020-03-31,1400.0,0.0,0.0
2090533,2020-03-31 23:30:00,2020-03-31,1410.0,0.0,0.0
2090543,2020-03-31 23:40:00,2020-03-31,1420.0,0.0,0.0
2090553,2020-03-31 23:50:00,2020-03-31,1430.0,0.0,0.0


In [ ]:
calc_R(AWS_DATA_RN, 'rn')

0.7056586044678055


-468.665148605295

In [ ]:
def plot_interpolation(df, target, date):
  df_date = df[df['date'] == date]
  x = df_date['time']
  y = df_date[target]
  plt.plot(x, y)

In [ ]:
import datetime
date = datetime.datetime.strptime("2016-04-01 00:00:00", '%Y-%m-%d %H:%M:%S').date()
plot_interpolation(AWS_DATA, 'ta', date)

NameError: ignored